In [33]:
import time
import random
import string

In [10]:
def foo_time(foo, *args, n_iterations=100):
    min_time = float('inf')
    for _ in range(n_iterations):
        t0 = time.perf_counter()
        foo(*args)
        t1 = time.perf_counter()
        min_time = min(min_time, t1 - t0)
    return min_time

# Fractional knapsack

The first line contains the number of items $1 ≤ n ≤ 10^3$ and the capacity of the knapsack $0≤W≤2⋅10^6$. Each of the next $n$ lines specifies the value $0≤c_i≤2⋅10^6$ and the weight $0<w_i≤2⋅10^6$ of the item ($n$, $W$, $c_i$, $w_i$ are integers). Return max value of items parts (any item may be fractioned with proportional value/weight decrease) may be placed in this knapsack.

Sample Input:
```mermaid
3 50
60 20
100 50
120 30
```

Sample Output:
```mermaid
180.000
```

<p>Time Limit: 5 секунд</p>
<p>Memory Limit: 256 MB</p>

In [3]:
preset = list(map(int, input().split()))
inp_list = []
for _ in range(preset[0]):
    inp_list.append(list(map(int, input().split())))

def fractional_knapsack(knapsack_size, items):
    res_free_size = knapsack_size
    sorted_input = sorted(items, key=lambda x: x[0]/x[1], reverse=True)
    res = 0

    for el in sorted_input:
        if res_free_size <= 0:
            break
        res += min(el[1], res_free_size) * el[0]/el[1]
        res_free_size -= min(el[1], res_free_size)
    return res

print(fractional_knapsack(preset[1], inp_list))

180.0


In [4]:
def fractional_knapsack_test():
    assert fractional_knapsack (0, [(60, 20)]) == 0.0
    assert fractional_knapsack(25, [(60, 20)]) == 60.0
    assert fractional_knapsack(25, [(60, 20), (0, 100)]) == 60.0
    assert fractional_knapsack(25, [(60, 20), (50, 50)]) == 60.0 + 5.0
    assert fractional_knapsack(50, [(60, 20), (100, 50), (120, 30)]) == 180.0

    for attempt in range(100):
        n = random.randint(1, 1000)
        capacity = random.randint(0, 2 * 10**6)
        items = []
        for i in range(n):
            weight = random.randint(1, 2 * 10 ** 6)
            value = random.randint(0, 2 * 10 ** 6)
            items.append((value, weight))
        t = foo_time(fractional_knapsack, capacity, items)
        assert t < 5
fractional_knapsack_test()

# Partition problem

By a given number $1≤n≤10^9$, find the maximum number $k$ for which $n$ can be represented as the sum of $k$ unique natural terms. Print the number $k$ in the first line, $k$ terms in the second line.

Sample Input 1:
```mermaid
4
```
Sample Output 1:
```mermaid
2
1 3
``` 
Sample Input 2:
```mermaid
6
```
Sample Output 2:
```mermaid
3
1 2 3 
```
<p>Time Limit: 5 секунд</p>
<p>Memory Limit: 256 MB</p>

In [6]:
n = 6

def create_partition(n):
    saved_numbers = []
    sum_accum = 0
    while len(saved_numbers) == 0 or sum_accum < n:

        if sum_accum + (len(saved_numbers) + 1)*2 >= n:
            last_item = len(saved_numbers) + 1
            for i in range(last_item, last_item*2 + 1):
                if sum_accum + i >= n:
                    saved_numbers.append(i)
                    sum_accum += i
                    break
        else:
            sum_accum += len(saved_numbers) + 1
            saved_numbers.append(len(saved_numbers) + 1)

    return saved_numbers

partition = create_partition(n)
print(len(partition))
print(' '.join(list(map(str,partition))))

3
1 2 3


In [15]:
def partition_test():
    for attempt_idx in range(100):
        n = random.randint(1, 10**9)
        t = foo_time(create_partition, n, n_iterations=10)
        assert t < 5
partition_test()

# Huffman encoding

By a given non-empty string $s$ of length at most $10^4$, consisting of lowercase latin letters, build an optimal prefix-free code. In the first line print the number of distinct letters $k$ occurring in the string and the size of the resulting encoded string. In the next $k$ lines write the codes of the letters in the format "letter: code". In the last line print the whole encoded string.

Sample Input 1:
```mermaid
a
```
Sample Output 1:
```mermaid
1 1
a: 0
0
```
Sample Input 2:
```mermaid
abacabad
```
Sample Output 2:
```mermaid
4 14
a: 0
b: 10
c: 110
d: 111
01001100100111
```

In [31]:
class Huffman_node(object):
    def __init__(self, label, freq, left_child=None, right_child=None, parent=None):
        self.left_child = left_child
        self.right_child = right_child
        self.parent = parent
        self.freq = freq
        self.label = label
        
    def get_letter_codes(self, letter2code, res='', depth=0):

        if self.left_child is None and self.right_child is None :
            if depth > 0:
                letter2code[self.label] = res
            else:
                letter2code[self.label] = res+'0'
                
        if self.left_child is not None:
            self.left_child.get_letter_codes(letter2code, res+'1', depth+1)

        if self.right_child is not None:
            self.right_child.get_letter_codes(letter2code, res+'0', depth+1)
            
def pop_min_pair(node_dict):
    min_freq = float('inf')
    min_freq_key = ''
    
    for key in node_dict.keys():
        if min_freq >= node_dict[key].freq:
            min_freq = node_dict[key].freq
            min_freq_key = key
            
    return (min_freq_key, node_dict.pop(min_freq_key))

def encode(row, letter2code):
    res = ''
    for char in row:
        res += letter2code[char]
    return res

    
def get_huffman_code(row):
    keys = sorted(set(row))
    vocab = {}

    for key in keys:
        vocab[key] = row.count(key)
    
    node_dict = {key: Huffman_node(key, vocab[key]) for key in vocab.keys()}


    for i in range(len(node_dict)-1):
        left_items = pop_min_pair(node_dict)
        right_items = pop_min_pair(node_dict)
        
        node_dict[left_items[0]+right_items[0]] = Huffman_node(
            left_items[0] + right_items[0],
            left_items[1].freq + right_items[1].freq, 
            left_child=left_items[1], 
            right_child=right_items[1]
            )

    node = node_dict.popitem()[1]

    letter2code = {}
    node.get_letter_codes(letter2code)

    code = encode(row, letter2code)
    res_str = f"{len(set(row))} {len(code)}\n" + '\n'.join([f'{letter}: {letter2code[letter]}' for letter in set(row)]) + "\n" + f"{code}"
    return res_str
row = 'abacabad'
huffman_code = get_huffman_code(row)
print(huffman_code)

4 14
c: 110
b: 10
a: 0
d: 111
01001100100111


# Huffman decoding

Restore the original string by its code and unprefixed Huffman code.

The first line of the input file contains two space-separated integers $k$ and $l$ — the number of different letters in the string and the size of the resulting encoded string, respectively. The next $k$ lines contain letter codes in the "letter: code" format. Neither code is a prefix of another. The letters can be listed in any order. Only lowercase letters of the Latin alphabet can be used as letters; each of these letters occurs at least once in the string. Finally, the last line contains the encoded string. The original string and codes of all letters are non-empty. The given code is such that the encoded string has the smallest possible size.

In the first line of the output file print the string $s$. It must consist of lowercase letters of the Latin alphabet. It is guaranteed that the length of the correct answer does not exceed $10^4$ characters.


Sample Input 1:
```pyhon
1 1
a: 0
0
```
Sample Output 1:
```pyhon
a
```
Sample Input 2:
```pyhon
4 14
a: 0
b: 10
c: 110
d: 111
01001100100111
```
Sample Output 2:
```pyhon
abacabad
```

In [34]:

class Node(object):
    def __init__(self, label, left_child=None, right_child=None):
        self.left_child = left_child
        self.right_child = right_child
        self.label = label
    
def get_huffman_decode(huffman_code):
    inp = huffman_code.splitlines()
    preset = inp[0]

    n_letters = int(list(preset.split())[0])
    key_code_pair_list = []
    for inp_pair in inp[1:-1]:
        key_code_pair_list.append(inp_pair.split(': '))

    inp_code = inp[-1]
    sorted_items = sorted(key_code_pair_list, key=lambda x: len(x[1]))
    sorted_labels = [list(item)[0] for item in sorted_items]
    sorted_codes = [list(item)[1] for item in sorted_items]

    root_node = Node('')

    for label, code in zip(sorted_labels, sorted_codes):
        current_node = root_node
        for direction in code:
            new_node = Node('')
            if direction == '0':
                if current_node.left_child is None:
                    current_node.left_child = new_node
                else:
                    new_node = current_node.left_child
            if direction == '1':
                if current_node.right_child is None:
                    current_node.right_child = new_node
                else:
                    new_node = current_node.right_child
            current_node = new_node
        current_node.label = label

    current_node = root_node
    res = ''
    for direction in inp_code:
        if direction == '0':
            current_node = current_node.left_child
        if direction == '1':
            current_node = current_node.right_child
        if current_node.label != '':
            res += current_node.label
            current_node = root_node
    return res

get_huffman_decode(huffman_code)

'abacabad'

In [38]:
def huffman_test(n_iter=100):
    for i in range(n_iter):
        seq_len = random.randint(1, 32)
        src_seq = "".join(random.choice(string.ascii_letters) for _ in range(seq_len))
        code = get_huffman_code(src_seq)
        out_seq = get_huffman_decode(code)
        assert src_seq == out_seq

huffman_test()

# Priority queue

First input line contains the number of operations. Each of the next $n$ lines sets up an operation one of the following kinds:

* Insert $x$, where $0≤x≤10^9$ — is an integer number;
* ExtractMax.

First operation adds number $x$ to the priority queue, second - extracts maximum number and prints it.

Sample Input:
```mermaid
6
Insert 200
Insert 10
ExtractMax
Insert 5
Insert 500
ExtractMax
```
Sample Output:
```mermaid
200
500
```

In [58]:
import sys
from abc import ABC, abstractmethod

class PriorityQueue(ABC):
    @abstractmethod
    def __init__(self):
        pass
    @abstractmethod
    def insert(self, p):
        pass
    @abstractmethod
    def get_root(self):
        pass
    @abstractmethod
    def pop_heap(self):
        pass


class MaxHeapQueue(PriorityQueue):
    def __init__(self, base=2):
        self.data = []
        self.base = base
    
    def _get_max_children_idx(self, idx):
        if idx * self.base > len(self.data):
            return None
        left_child_idx = min(len(self.data), idx*self.base)
        right_child_idx = min(len(self.data), idx*self.base+1)
        if self.data[left_child_idx - 1] >= self.data[right_child_idx - 1]:
            return left_child_idx
        else:
            return right_child_idx


    def _get_parent_idx(self, idx):
        return idx//self.base

    def _descend_element(self, idx):
        max_child_idx = self._get_max_children_idx(idx)
        if max_child_idx is not None and self.data[idx - 1] <= self.data[max_child_idx - 1] and idx < len(self.data):
            tmp = self.data[max_child_idx - 1]
            self.data[max_child_idx - 1] = self.data[idx - 1]
            self.data[idx - 1] = tmp
            self._descend_element(max_child_idx)


    def _ascend_element(self, idx):
        parent_idx = self._get_parent_idx(idx)
        if self.data[parent_idx - 1] <= self.data[idx - 1] and idx > 1 and parent_idx > 0:
            tmp = self.data[parent_idx - 1]
            self.data[parent_idx - 1] = self.data[idx - 1]
            self.data[idx - 1] = tmp
            self._ascend_element(parent_idx)

    
    def get_root(self):
        return self.data[0]

    def pop_heap(self):
        res = self.data[0]
        if len(self.data) > 1:
            self.data[0] = self.data.pop()
            self._descend_element(1)
        else:
            self.data.pop()
        return res
    
    def insert(self, el):
        self.data.append(el)
        self._ascend_element(len(self.data))

n_commands = int(input())
command_list = []
for _ in range(n_commands):
    command_list.append(sys.stdin.readline().split())

pqueue = MaxHeapQueue()
for command in command_list:
    if command[0] == 'Insert':
        pqueue.insert(int(command[1]))
    if command[0] == 'ExtractMax':
        print(pqueue.pop_heap())

In [112]:
import heapq
import numpy as np

In [108]:
class MaxHeapStd(PriorityQueue):
    def __init__(self, data: list = []) -> None:
        self.heap = data.copy()
        self.heap = list(map(lambda x: -x, self.heap))
        heapq.heapify(self.heap)
        
    def insert(self, p):
        heapq.heappush(self.heap, -p)
    
    def get_root(self):
        return -self.heap[0]
    
    def pop_heap(self):
        return -heapq.heappop(self.heap)


In [152]:
def heap_test(n_iter=1):
    for iter_idx in range(n_iter):
        n_operations = random.randint(0, 10**5)
        std_out_log = []
        custom_out_log = []
        custom_queue = MaxHeapQueue()
        std_queue = MaxHeapStd()
        # 0 - cmd for inserting
        # 1 - cmd for popping
        commands = np.random.binomial(1, .75, size=n_operations)
        elements_inserted = 0
        for cmd in commands:
            is_pop = bool(cmd)
            if is_pop and elements_inserted:
                custom_out_log.append(custom_queue.pop_heap())
                std_out_log.append(std_queue.pop_heap())
                elements_inserted -= 1
            elif not is_pop:
                new_element = random.randint(0, 10**9)
                custom_queue.insert(new_element)
                std_queue.insert(new_element)
        assert std_out_log == custom_out_log

heap_test(100)